
## Bonus Task: word2vec in PyTorch

As you may have already noticed, the idea behind [word2vec](https://arxiv.org/pdf/1310.4546) is quite general. In this task, you will implement it yourself.

Disclaimer: don't be surprised if the implementation from gensim (or similar libraries) trains faster and is more accurate. It uses many optimizations and accelerations, along with quite efficient code. Your task is to achieve intermediate results within a reasonable time.

P.S. Strangely enough, we won't need a GPU for this task.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk bokeh umap-learn```


In [ ]:
#!pip install --upgrade nltk bokeh umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.4 MB/s eta 0:00:00


In [ ]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2025-01-13 00:15:45--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2025-01-13 00:15:46--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc54bfa53b800cd8bb964c8653e5.dl.dropboxusercontent.com/cd/0/inline/CiH0CMahWGOtAzLumNR3STX201Eu75oLShAzdQb0A4BW0enJ4ZA9xKdanIEs8V_282wlKIk8x48Qq_3ZMe9fXmqLdX_FyRvoz5CdVEm_-QUMctotvhiNb5_M51OEgDWF8jw/file?dl=1# [following]
--2025-01-13 00:15:46--  https://uc54bfa53b800cd8bb964c8653e5.dl.dropboxusercontent.com/cd/0/inline/CiH0CMah

In [ ]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"


Tokenization is the first step.
The texts we work with include punctuation, emoticons, and other non-standard tokens, so simple `str.split` won't work.

Let's turn to `nltk` - a library that has found wide application in the field of NLP.

In [ ]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [ ]:
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
]
data_tok = [x for x in data_tok if len(x) >= 3]

Some checks:

In [ ]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

Define some constants: window size  for skip-gram model.

In [ ]:
min_count = 5
window_radius = 5

In [ ]:
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if counter >= min_count:
        word_count_dict[word] = counter

vocabulary = set(word_count_dict.keys())
del vocabulary_with_counter

In [ ]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {index: word for word, index in word_to_index.items()}

In [ ]:
index_to_word[10]

'town'

Pairs `(word, context)` are generated on the base of our data.

In [ ]:
context_pairs = []

for text in data_tok:
    for i, central_word in enumerate(text):
        context_indices = range(
            max(0, i - window_radius), min(i + window_radius, len(text))
        )
        for j in context_indices:
            if j == i:
                continue
            context_word = text[j]
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append(
                    (word_to_index[central_word], word_to_index[context_word])
                )

print(f"Generated {len(context_pairs)} pairs of target and context words.")

Generated 40220313 pairs of target and context words.


In [ ]:
context_pairs[0]

(19386, 24381)

#### Subtask 1: Subsampling
To smooth out the differences in word frequencies, you need to implement a subsampling mechanism.
For this, you need to implement the function below.

The probability of **excluding** a word from training (at a fixed step) is calculated as:
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
where $f(w_i)$ is the normalized frequency of the word, and $t$ is the given threshold.

In [ ]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """

    ### YOUR CODE HERE
    # Total word count in the dataset
    total_count = sum(word_count_dict.values())

    # Compute normalized frequencies
    word_freq_dict = {word: count / total_count for word, count in word_count_dict.items()}

    # Compute probabilities of keeping each word
    keep_prob_dict = {
        word: min(1, (threshold / freq) ** 0.5 + (threshold / freq))
        for word, freq in word_freq_dict.items()
    }
    # keep_prob_dict = {
    #     word: 0 for word in word_count_dict.keys()
    # }  # THIS IS A PLACEHOLDER!
    return keep_prob_dict

In [ ]:
np.sqrt((1e-5)*6050/5000) #((1e-5/5000*(5000+1000+50))**(-0.5) + 1)* 1e-5/5000*(5000+1000+50)

0.0034785054261852176

In [ ]:
word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
subsample_frequent_words(word_counts)

{'the': 0.0034906054261852177,
 'is': 0.007838674593052023,
 'apple': 0.03599505426185218}

#### Subtask 2: Negative Sampling
For more efficient training, it is necessary not only to predict high probabilities for words in the context but also to predict low probabilities for words not seen in the context. To achieve this, you need to calculate the probability of using a word as a negative sample by implementing the function below.

In the original paper, it is suggested to evaluate the probability of a word serving as a negative sample according to the distribution
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$
where $U(w)$ is the word distribution by frequency (or, as it is sometimes called, the unigram distribution), and $Z$ is the normalization constant to ensure the total measure equals 1.

In [ ]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """

    ### YOUR CODE HERE
    # Step 1: Raise counts to the power of 0.75
    adjusted_frequencies = {word: count ** 0.75 for word, count in word_count_dict.items()}

    # Step 2: Compute the total sum of adjusted frequencies
    total_adjusted_frequency = sum(adjusted_frequencies.values())

    # Step 3: Normalize to create a probability distribution
    negative_sampling_prob_dict = {
        word: freq / total_adjusted_frequency
        for word, freq in adjusted_frequencies.items()
    }
    # negative_sampling_prob_dict = {
    #     word: 0 for word in word_count_dict.keys()
    # }  # THIS IS A PLACEHOLDER!
    return negative_sampling_prob_dict

For convenience, let's convert the obtained dictionaries into arrays (since all words are already numbered anyway).

In [ ]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

In [ ]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

In [ ]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

If everything went successfully, the function below will help you generate sub-samples (batches).

In [ ]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    batch = []
    neg_samples = []

    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [ ]:
batch_size = 4
num_negatives = 15
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

Finally, it's time to implement the model. We would like to point out that using linear layers (`nn.Linear`) is not always justified!

Let us remind you that in the case of negative sampling, the task is to maximize the following functional:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$
where
- $\mathbf{v}_{w_I}$ --- centeral word vector,
- $\mathbf{v}'_{w_O}$ --- context vector,
- $k$ --- number of negative samples,
- $P_n(w)$ --- negative samples distribution,
- $\sigma$ --- sigmoid function.

In [ ]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        # Center word embeddings (lookup table)
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # Context word embeddings (lookup table)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center_words, pos_context_words, neg_context_words):
        # YOUR CODE HERE
        # Get embeddings for center words
        center_embeds = self.center_embeddings(center_words)  # (B, D)

        # Get embeddings for positive context words
        pos_context_embeds = self.context_embeddings(pos_context_words)  # (B, D)

        # Get embeddings for negative context words
        neg_context_embeds = self.context_embeddings(neg_context_words)  # (B, K?, D)

        pos_scores = torch.sum(center_embeds * pos_context_embeds, dim=1) # 0  # THIS IS A PLACEHOLDER
        neg_scores = torch.bmm(
            neg_context_embeds, center_embeds.unsqueeze(2)
        ).squeeze(2)  # (B, K) #torch.sum(center_embeds * neg_context_embeds, dim=1) # 0  # THIS IS A PLACEHOLDER

        return pos_scores, neg_scores

In [ ]:
device = torch.device("cpu")

In [ ]:
vocab_size = len(word_to_index)
embedding_dim = 32
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [ ]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()
        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [ ]:
steps = 1100
batch_size = 512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

  0%|          | 0/1100 [00:00<?, ?it/s]

Step 0, Loss: 1.514573335647583, learning rate: [0.00015625]
Step 100, Loss: 1.5109843397140503, learning rate: [0.00015625]
Step 200, Loss: 1.508030996322632, learning rate: [7.8125e-05]
Step 300, Loss: 1.5155027532577514, learning rate: [7.8125e-05]
Step 400, Loss: 1.5214697110652924, learning rate: [7.8125e-05]
Step 500, Loss: 1.4963977825641632, learning rate: [3.90625e-05]
Step 600, Loss: 1.5041557335853577, learning rate: [3.90625e-05]


KeyboardInterrupt: 

In [ ]:
torch.save(model, 'model.pth')

Finally, use the obtained weight matrix as the matrix for word vector representations. We recommend using the matrix that corresponds to the context words (i.e., the decoder) for submission.

In [ ]:
_model_parameters = model.parameters()
embedding_matrix_center = next(
    _model_parameters
).detach()  # Assuming that first matrix was for central word
embedding_matrix_context = next(
    _model_parameters
).detach()  # Assuming that second matrix was for context word

In [ ]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

Simple checks:

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("dell", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

AssertionError: 

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

Finally, let's take a look at the words closest by cosine similarity. The function is implemented below.

In [ ]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [ ]:
find_nearest("python", embedding_matrix_context, k=10)

[('starvation', 0.6635692119598389),
 ('java', 0.6666164994239807),
 ('sector', 0.667748749256134),
 ('academys', 0.6769938468933105),
 ('centres', 0.6844515800476074),
 ('mla', 0.6865198612213135),
 ('aw', 0.6891686916351318),
 ('companies', 0.6957893371582031),
 ('llb', 0.7053821682929993),
 ('python', 0.9999999403953552)]


You can also visually check how frequently occurring words are represented in the latent space.

In [ ]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

In [ ]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [ ]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

In [ ]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

figure(id='p1106', ...)